In [41]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [42]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6934, 7)

In [43]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6643, 5)

In [44]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6940, 9)

fin empresas y comienzo de rescate de pacientes


In [45]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1629586, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [46]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1194147, 4)

In [47]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1194757, 12)

In [48]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [49]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [40]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [41]:
#pacientes_empresas22.shape

In [42]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [43]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [44]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [26]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Mercer"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [27]:
Pacientes_Archivo.head(10)

,RUT TITULAR,rut1,TIPO,NOMBRE,Unnamed: 4,nombres,apellidos,FEC.NACIMIENTO,Unnamed: 8,Unnamed: 9,Unnamed: 10,nacimiento,email,celular
0,10111013-3,10111013-3,ASEGURADO,RICARDO MORALES GARCIA,NaN,RICARDO,MORALES GARCIA,26/11/1964,26,11,1964,1964-11-26,rimoga0126@gmail.com,990896880
1,10111013-3,12241381-0,CARGA NORMAL - Conyuge,LORENA PINO MORENO,NaN,LORENA,PINO MORENO,01/11/1972,1,11,1972,1972-11-01,NaN,NaN
2,11661987-3,11661987-3,ASEGURADO,ALEJANDRO JIMENEZ MORALES,NaN,ALEJANDRO,JIMENEZ MORALES,21/12/1969,21,12,1969,1969-12-21,alejojimenez1969@gmail.com,995196898
3,13027325-4,13027325-4,ASEGURADO,JUAN CRISTOBAL RECART SALAS,NaN,JUAN,CRISTOBAL RECART SALAS,26/06/1976,26,6,1976,1976-06-26,JCRECART100@GMAIL.COM,NaN
4,13027325-4,13270895-9,CARGA NORMAL - Conyuge,CLAUDIA HIDALGO FLORES,NaN,CLAUDIA,HIDALGO FLORES,11/04/1977,11,4,1977,1977-04-11,NaN,NaN
5,13027325-4,21527286-9,CARGA NORMAL - Hijo(a),MARIA DE LOS ANGELES RECONT HIDALGO,NaN,MARIA,DE LOS ANGELES RECONT HIDALGO,19/07/2004,19,7,2004,2004-07-19,NaN,NaN
6,13027325-4,21678628-9,CARGA NORMAL - Hijo(a),MARIA GRACIA RECONT HIDALGO,NaN,MARIA,GRACIA RECONT HIDALGO,10/01/2006,10,1,2006,2006-01-10,NaN,NaN
7,13027325-4,23512105-0,CARGA NORMAL - Hijo(a),MARIA ESPERANZA DE JESUS RECART HIDALGO,NaN,MARIA,ESPERANZA DE JESUS RECART HIDALGO,26/12/2010,26,12,2010,2010-12-26,NaN,NaN
8,13027325-4,24093347-0,CARGA NORMAL - Hijo(a),JOSE PEDRO RECART HIDALGO,NaN,JOSE,PEDRO RECART HIDALGO,19/10/2012,19,10,2012,2012-10-19,NaN,NaN
9,13657949-5,13657949-5,ASEGURADO,MACARENA ANDREA BEVILACQUA SALAS,NaN,MACARENA,ANDREA BEVILACQUA SALAS,23/08/1979,23,8,1979,1979-08-23,mabesa23@gmail.com,966462521


In [28]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [29]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,RUT TITULAR,rut1,TIPO,NOMBRE,Unnamed: 4,nombres,apellidos,FEC.NACIMIENTO,Unnamed: 8,Unnamed: 9,Unnamed: 10,nacimiento,email,celular,rut
0,10111013-3,10111013-3,ASEGURADO,RICARDO MORALES GARCIA,NaN,RICARDO,MORALES GARCIA,26/11/1964,26,11,1964,1964-11-26,rimoga0126@gmail.com,990896880,101110133
1,10111013-3,12241381-0,CARGA NORMAL - Conyuge,LORENA PINO MORENO,NaN,LORENA,PINO MORENO,01/11/1972,1,11,1972,1972-11-01,NaN,NaN,122413810
2,11661987-3,11661987-3,ASEGURADO,ALEJANDRO JIMENEZ MORALES,NaN,ALEJANDRO,JIMENEZ MORALES,21/12/1969,21,12,1969,1969-12-21,alejojimenez1969@gmail.com,995196898,116619873
3,13027325-4,13027325-4,ASEGURADO,JUAN CRISTOBAL RECART SALAS,NaN,JUAN,CRISTOBAL RECART SALAS,26/06/1976,26,6,1976,1976-06-26,JCRECART100@GMAIL.COM,NaN,130273254
4,13027325-4,13270895-9,CARGA NORMAL - Conyuge,CLAUDIA HIDALGO FLORES,NaN,CLAUDIA,HIDALGO FLORES,11/04/1977,11,4,1977,1977-04-11,NaN,NaN,132708959


In [30]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_3712\485331713.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,RUT TITULAR,rut1,TIPO,NOMBRE,Unnamed: 4,nombres,apellidos,FEC.NACIMIENTO,Unnamed: 8,Unnamed: 9,Unnamed: 10,nacimiento,email,celular,rut
0,10111013-3,10111013-3,ASEGURADO,RICARDO MORALES GARCIA,NaN,RICARDO,MORALES GARCIA,26/11/1964,26,11,1964,1964-11-26,rimoga0126@gmail.com,990896880,101110133
1,10111013-3,12241381-0,CARGA NORMAL - Conyuge,LORENA PINO MORENO,NaN,LORENA,PINO MORENO,01/11/1972,1,11,1972,1972-11-01,NaN,NaN,122413810
2,11661987-3,11661987-3,ASEGURADO,ALEJANDRO JIMENEZ MORALES,NaN,ALEJANDRO,JIMENEZ MORALES,21/12/1969,21,12,1969,1969-12-21,alejojimenez1969@gmail.com,995196898,116619873
3,13027325-4,13027325-4,ASEGURADO,JUAN CRISTOBAL RECART SALAS,NaN,JUAN,CRISTOBAL RECART SALAS,26/06/1976,26,6,1976,1976-06-26,JCRECART100@GMAIL.COM,NaN,130273254
4,13027325-4,13270895-9,CARGA NORMAL - Conyuge,CLAUDIA HIDALGO FLORES,NaN,CLAUDIA,HIDALGO FLORES,11/04/1977,11,4,1977,1977-04-11,NaN,NaN,132708959


In [31]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_3712\2118328728.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_3712\2118328728.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [32]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(56, 25)

In [33]:
Pacientes_con_id.head()

,RUT TITULAR,rut1,TIPO,NOMBRE,Unnamed: 4,nombres,apellidos,FEC.NACIMIENTO,Unnamed: 8,Unnamed: 9,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,10111013-3,10111013-3,ASEGURADO,RICARDO MORALES GARCIA,NaN,Ricardo,Morales Garcia,26/11/1964,26,11,...,True,0018c00002eWmqSAAS,101110133,Ricardo Morales Garcia,RUT,None,None,None,Hombre,None
1,10111013-3,12241381-0,CARGA NORMAL - Conyuge,LORENA PINO MORENO,NaN,Lorena,Pino Moreno,01/11/1972,1,11,...,True,0018c00002eWWnEAAW,122413810,Lorena Pino Moreno,RUT,None,None,None,Mujer,None
2,11661987-3,11661987-3,ASEGURADO,ALEJANDRO JIMENEZ MORALES,NaN,Alejandro,Jimenez Morales,21/12/1969,21,12,...,True,0018c00002eWmg5AAC,116619873,Alejandro Jimenez Morales,RUT,None,None,None,Hombre,None
3,13027325-4,13027325-4,ASEGURADO,JUAN CRISTOBAL RECART SALAS,NaN,Juan,Cristobal Recart Salas,26/06/1976,26,6,...,True,0018c00002eWmqfAAC,130273254,Juan Cristobal Recart Salas,RUT,None,None,None,Hombre,None
4,13027325-4,13270895-9,CARGA NORMAL - Conyuge,CLAUDIA HIDALGO FLORES,NaN,Claudia,Hidalgo Flores,11/04/1977,11,4,...,True,0018c00002eWVWKAA4,132708959,Claudia Hidalgo Flores,RUT,None,None,None,Mujer,None


In [34]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"].astype(str).str.upper()
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"].apply(str).str.strip().replace("+","")
    Subir_pacientes['Phone'] = Subir_pacientes['Phone'].str.title().replace("NaN", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtsEAE'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,Phone,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
9,False,Macarena,Andrea Bevilacqua Salas,136579495,mabesa23@gmail.com,1979-08-23,966462521,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
10,False,Daniel,Eduardo Araya Alvarez,120269631,,1978-10-11,Nan,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
11,False,Elena,Araya Bevilacqua,267594368,,2019-03-22,Nan,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
16,False,Roberta,Nini Nunez Diaz,14131243K,robertanunezd@gmail.com,1980-06-19,962033429,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
17,False,Rafael,Enrique Tobia Diaz,257651843,,1980-08-27,Nan,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [35]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [36]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A


In [37]:
campaña_seleccionada.shape

(0, 12)

In [38]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [39]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [40]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
